In [1]:
%pip install holoviews


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install networkx


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import requests
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts
from bs4 import BeautifulSoup
from urllib.request import urljoin


In [4]:
sites = "https://www.google.com/" #@param {type:"string"}
depth = 3 #@param {type:"slider", min:1, max:10, step:1}
max_child = 10 #@param {type:"slider", min:1, max:100, step:1}

def crawl(sites, depth=1, current_depth=0, max_child=10):
    get_links = lambda site: pd.Series(urljoin(site, tag.get('href')) for tag in BeautifulSoup(requests.get(site).text).find_all('a')).drop_duplicates().head(max_child)
    
    sites = pd.concat([sites,pd.DataFrame([{'link': link, 'parent': site, 'depth': current_depth + 1}
                                       for site in sites[sites.depth == current_depth].link
                                       for link in get_links(site)])], 
                         ignore_index=True).drop_duplicates('link')
    
    if depth > current_depth + 1:
        return crawl(sites, depth=depth, current_depth=current_depth+1, max_child=max_child)
    else:
        return sites


result = crawl(sites=pd.DataFrame([{'link': site, 'depth': 0, 'parent': site} for site in sites.split()]), 
               max_child=max_child,
               depth=depth)

In [5]:
result

,link,depth,parent
0,https://www.google.com/,0,https://www.google.com/
1,https://www.google.com/imghp?hl=km&tab=wi,1,https://www.google.com/
2,https://maps.google.com.kh/maps?hl=km&tab=wl,1,https://www.google.com/
3,https://play.google.com/?hl=km&tab=w8,1,https://www.google.com/
4,https://news.google.com/?tab=wn,1,https://www.google.com/
...,...,...,...
538,https://support.google.com/websearch/?hl=en,3,https://support.google.com/websearch/?p=displa...
539,https://support.google.com/websearch/community...,3,https://support.google.com/websearch/?p=displa...
542,https://support.google.com/websearch/?p=result...,3,https://support.google.com/websearch/?p=result...
546,https://accounts.google.com/ServiceLogin?hl=en...,3,https://support.google.com/websearch/?p=result...


In [6]:
# this is just the example scenarios
# from the results you can see the links "https://support.google.com/websearch", "https://play.google.com"
# it could be the all services google site

In [9]:
import re

html = """<html>...</html>""" # replace this with the actual HTML content
pattern = re.compile(r'<a href="(?P<url>[^"]*)">(?P<service>[^<]*)</a>')
services = []

for match in pattern.finditer(html):
    service = {'url': match.group('url'), 'name': match.group('service')}
    services.append(service)

for service in services:
    print(f"Name: {service['name']}, URL: {service['url']}")


In [10]:
#Task : Visualize the services : - how many services that google have?
# extract the links from the result dataframe
links = result['link'].tolist()

# extract the unique domains from the links
domains = set([link.split('/')[2] for link in links])

# count the number of unique domains
num_services = len(domains)

# print the number of services
print(f"Google has {num_services} services.")




Google has 24 services.
